In [1]:
# Load necessaries libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the CSV file into a DataFrame
data = pd.read_csv('NYPD_Complaint_Data.csv')



In [2]:
# Check the dimensions of the DataFrame
print("Dimensions of the dataset:", data.shape)

# List the columns
print(data.columns)

# Check the data types of each column
print("\nData types of columns:")
print(data.dtypes)




Dimensions of the dataset: (1968368, 35)
Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD',
       'PD_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM',
       'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'JURIS_DESC', 'JURISDICTION_CODE',
       'PARKS_NM', 'HADEVELOPT', 'HOUSING_PSA', 'X_COORD_CD', 'Y_COORD_CD',
       'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT',
       'Latitude', 'Longitude', 'Lat_Lon', 'PATROL_BORO', 'STATION_NAME',
       'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX'],
      dtype='object')

Data types of columns:
CMPLNT_NUM            object
CMPLNT_FR_DT          object
CMPLNT_FR_TM          object
CMPLNT_TO_DT          object
CMPLNT_TO_TM          object
ADDR_PCT_CD          float64
RPT_DT                object
KY_CD                  int64
OFNS_DESC             object
PD_CD                float64
PD_DESC               object
CRM_ATPT_CPTD_CD      object
LAW_CA

In [3]:
# Remove unwanted columns
cool_columns = ['CMPLNT_NUM','CMPLNT_FR_DT', 'CMPLNT_FR_TM','OFNS_DESC','PD_DESC', 'Latitude', 'Longitude']
data = data[cool_columns]

# Rename columns
data.columns = ['id', 'date', 'time', 'offense_type', 'offense_desc', 'Latitude', 'Longitude']
print(data.head())

# Format offence_type and offense_desc
data['offense_desc'] = data['offense_desc'].str.title()
data['offense_type'] = data['offense_type'].str.title()



          id        date      time     offense_type  \
0  188272853  10/02/2002  15:00:00  DANGEROUS DRUGS   
1  188382939  01/01/2015  08:00:00       SEX CRIMES   
2  188275801  04/03/2015  13:00:00    PETIT LARCENY   
3  188336017  10/24/2015  19:00:00       SEX CRIMES   
4  188251213  02/01/2016  21:00:00    GRAND LARCENY   

                                   offense_desc   Latitude  Longitude  
0                CONTROLLED SUBSTANCE, POSSESSI  40.720442 -74.006744  
1                              SEXUAL ABUSE 3,2  40.810352 -73.924942  
2                LARCENY,PETIT FROM BUILDING,UN  40.576646 -73.976480  
3                    CHILD, ENDANGERING WELFARE  40.743481 -73.874004  
4  LARCENY,GRAND BY OPEN CREDIT CARD (NEW ACCT)  40.844157 -73.900605  


In [4]:
# Lets determine zipcode & borough for complaints. 
import geopandas as gpd
from shapely.geometry import Point
# Convert the DataFrame to a GeoDataFrame with geometry
gdf = gpd.GeoDataFrame(
    data, 
    geometry=gpd.points_from_xy(data['Longitude'], data['Latitude']),
    crs="EPSG:4326"  # feet projection in nyc, see https://epsg.io/2263
)
# Load shapefile.
zip_shapefile = gpd.read_file('nyc_zips/ZIP_CODE_040114.shp')
borough_shapefile = gpd.read_file('nyc_boroughs/nybb.shp')


print(borough_shapefile.columns)
print(zip_shapefile.columns)

# Load shapefile and convert CRS
zip_shapefile = gpd.read_file('nyc_zips/ZIP_CODE_040114.shp').to_crs("EPSG:4326")
borough_shapefile = gpd.read_file('nyc_boroughs/nybb.shp').to_crs("EPSG:4326")

# Merge
merged_zips = gpd.sjoin(gdf, zip_shapefile, how='left', predicate='within')
merged_boroughs = gpd.sjoin(gdf, borough_shapefile, how='left', predicate='within')

# Merge the results back into the original DataFrame
data = data.merge(merged_zips[['ZIPCODE']], left_index=True, right_index=True, how='left')
data = data.merge(merged_boroughs[['BoroName']], left_index=True, right_index=True, how='left')



print(data.head())

# Check for NaN values
print("Rows with NaN in ZIPCODE:")
print(data[data['ZIPCODE'].isna()])

print("Rows with NaN in BoroName:")
print(data[data['BoroName'].isna()])

Index(['BoroCode', 'BoroName', 'Shape_Leng', 'Shape_Area', 'geometry'], dtype='object')
Index(['ZIPCODE', 'BLDGZIP', 'PO_NAME', 'POPULATION', 'AREA', 'STATE',
       'COUNTY', 'ST_FIPS', 'CTY_FIPS', 'URL', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry'],
      dtype='object')
          id        date      time     offense_type  \
0  188272853  10/02/2002  15:00:00  Dangerous Drugs   
1  188382939  01/01/2015  08:00:00       Sex Crimes   
2  188275801  04/03/2015  13:00:00    Petit Larceny   
3  188336017  10/24/2015  19:00:00       Sex Crimes   
4  188251213  02/01/2016  21:00:00    Grand Larceny   

                                   offense_desc   Latitude  Longitude ZIPCODE  \
0                Controlled Substance, Possessi  40.720442 -74.006744   10013   
1                              Sexual Abuse 3,2  40.810352 -73.924942   10454   
2                Larceny,Petit From Building,Un  40.576646 -73.976480   11224   
3                    Child, Endangering Welfare  40.743481 -73.874004 

In [5]:
# Save csv file
data.to_csv('NYPD_Complaint_Data_with_ZIP.csv', index=False)
